# Extreme Temperatures in Stuttgart from 2010 to 2019

Here is the Python code for plotting a line graph of the record high and record low temperatures by day of the year over the period 2010-2019 in the area of Stuttgart, Germany.

The area between the record high and record low temperatures for each day is shaded. A scatter of the 2019 data for any points (highs and lows) for which the ten year record (2010-2019) record high or record low was broken in 2019.

Leap days (i.e. February 29th) is removed for the purpose of this visualization.

The climate data used in this file comes from a subset of The National Centers for Environmental Information (NCEI) [Daily Global Historical Climatology Network](https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt) (GHCN-Daily). The GHCN-Daily is comprised of daily climate records from thousands of land surface stations across the globe.

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('data/2101475.csv')
data.head()

,STATION,NAME,LATITUDE,LONGITUDE,ELEVATION,DATE,PRCP,PRCP_ATTRIBUTES,SNWD,SNWD_ATTRIBUTES,TAVG,TAVG_ATTRIBUTES,TMAX,TMAX_ATTRIBUTES,TMIN,TMIN_ATTRIBUTES
0,GME00125338,"BAD LIEBENZELL, GM",48.7733,8.7294,352.0,2010-01-01,5.2,",,E",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,GME00125338,"BAD LIEBENZELL, GM",48.7733,8.7294,352.0,2010-01-02,0.5,",,E",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,GME00125338,"BAD LIEBENZELL, GM",48.7733,8.7294,352.0,2010-01-03,0.2,",,E",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,GME00125338,"BAD LIEBENZELL, GM",48.7733,8.7294,352.0,2010-01-04,0.0,",,E",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,GME00125338,"BAD LIEBENZELL, GM",48.7733,8.7294,352.0,2010-01-05,0.0,",,E",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# how many records?
len(data)

86942

In [4]:
# TMAX and TMIN are the columns we are going to use.
# drop lines where value of TAVG
data.dropna(subset = ['TMAX', 'TMIN'], inplace = True)

In [5]:
# how many records?
len(data)

51480

In [7]:
df_t = data.drop(columns=['PRCP', 'PRCP_ATTRIBUTES', 'SNWD', 
                          'SNWD_ATTRIBUTES', 'TAVG_ATTRIBUTES', 
                          'TMAX_ATTRIBUTES', 'TMIN_ATTRIBUTES'])
df_t.head()

,STATION,NAME,LATITUDE,LONGITUDE,ELEVATION,DATE,TAVG,TMAX,TMIN
2479,GME00126922,"OBERSULM WILLSBACH, GM",49.1289,9.3533,230.0,2010-01-01,NaN,6.1,-0.8
2480,GME00126922,"OBERSULM WILLSBACH, GM",49.1289,9.3533,230.0,2010-01-02,NaN,-0.7,-4.1
2481,GME00126922,"OBERSULM WILLSBACH, GM",49.1289,9.3533,230.0,2010-01-03,NaN,0.2,-3.2
2482,GME00126922,"OBERSULM WILLSBACH, GM",49.1289,9.3533,230.0,2010-01-04,NaN,-2.0,-4.5
2483,GME00126922,"OBERSULM WILLSBACH, GM",49.1289,9.3533,230.0,2010-01-05,NaN,-2.7,-5.1


In [12]:
import mplleaflet

# plot the stations on a map

lons = df_t.LONGITUDE.unique()
lats = df_t.LATITUDE.unique()

plt.figure(figsize=(8,8))

plt.scatter(lons, lats, c='r', alpha=0.7, s=200)

mplleaflet.display()